In [6]:
import nltk                   #Only show the difference
from operator import eq 

def OpinosisGraph(Z, G, PRI):
    for i in range(len(Z)):  
        for j in range(len(Z[i])):  
            LABEL = Z[i][j]
            PID = j
            SID = i
            if LABEL in G:
                PRI[LABEL].append((SID, PID))
            else:
                G[LABEL] = list()
                PRI[LABEL] = [(SID, PID)]
            if j > 0 and LABEL not in G[Z[i][j-1]]:
                G[Z[i][j-1]].append(Z[i][j])
                
def VSN(node):
    global PRI1
    if node[1] in ['.']:
        return False
    sigma_vsn = 15
    ele = PRI1[node]
    sum = 0               
    for i in range(len(ele)):
        sum = sum + ele[i][1]
    if sum/len(ele) > sigma_vsn:
        return False
    return True

def VEN(node):
    prohibit = ['and', 'but', 'for', 'nor', 'or', 'so', 'yet', ',', '.', '!', '?', ';',':',':']
    if(node[0] in prohibit):
        return True
    return False

def collapsible(node):
    if(node[1] == 'VERB'):
        return True
    else: 
        return False

    
def valid_sent(sent):                        
    all = ['VERB','.','ADJ','PRON','NOUN','ADV','PRON','CONJ','PRT','ADP','NUM','DET','X']
    count = dict()
    for i in sent:
        if i not in count:
            count[i] = 1
        else:
            count[i] += 1
    for i in count:
        if i[1] in ['ADJ','ADV','NOUN'] and count[i] > 1:
            return False
    #the first kind of valid sentence
    state = 0
    for i in sent:
        if state == 0 and i[1] == 'NOUN':
            state = 1
        if state == 1 and i[1] == 'VERB':
            state = 2
        if state == 2 and i[1] == 'ADJ':
            state = 3
    if state == 3:
        return True
    #the second kind of valid sentence
    state = 0
    for i in sent:
        if state == 0 and i[1] == 'ADJ':
            state = 2
        if state == 1 and i[1] == 'DET':  
            state = 2
        if state == 2 and i[1] == 'VERB':
            state = 3
    if state == 3:
        return True
    #the third kind of valid sentence
    state = 0
    for i in sent:
        if state == 0 and i[1] == 'ADV':
            state = 1
        if state == 1 and i[1] == 'ADJ':  
            state = 2
        if state == 2 and i[1] == 'NOUN':
            state = 3
    if state == 3:
        return True
    #the fourth kind of valid sentence
    state = 0
    for i in sent:
        if state == 0 and i[1] == 'ADV':
            state = 2
        #if state == 1 and i[1] == 'VERB':  #?
            #state = 2
        if state == 2 and i[1] == 'NOUN':
            state = 3
    if state == 3:
        return True
    return False
    

def check_inter(PRIa,PRIb):
    PRIc = []
    sigma_gap = 4
    for first in PRIa:
        for second in PRIb:
            if(first[0] == second[0] and abs(first[1]-second[1]) <= sigma_gap):
                PRIc.append(second)
                break
    return PRIc

def common_elements(list1, list2):
    count = 0
    for element in list1:
        if element in list2:
            count= count+1
    return count
                                            
def jaccard_fail(a,b):
    sigma_j = 0.8
    inter = common_elements(a,b)
    if ((inter/(len(a)+len(b)-inter)) >= sigma_j):
        return True
    if abs(inter-len(a)) < 3 or abs(inter-len(b)) < 3:
        return True
    return False
    

def eliminate_dup2(clist): 
    #print(clist)
    buf = []
    first = True
    for ele in clist:
        temp = []
        similar = False
        if(first):
            buf.append(ele)
            first = False
        else:
            for seen in buf:
                if jaccard_fail(seen[0],ele[0]):
                    similar = True
                    if ele[1]>seen[1]:     
                        temp.append(seen)
            if len(temp) != 0:  
                for seen in temp:
                    buf.remove(seen)
                buf.append(ele)
            elif similar == False:
                buf.append(ele) 
    return buf



def find_avg_score(clist):
    if len(clist)==0:
        return 0
    sum = 0
    for ele in clist:
        sum = sum + ele[1]
    return sum/len(clist)

def stitch_prev(Canchor,CC): 
    if len(CC) == 0:
        return Canchor
    CC = eliminate_dup2(CC) 
    winner = CC[0]
    for ele in CC:
        if ele[1] > winner[1]:
            winner = ele
    ans = []
    return (Canchor+winner[0])

def stitch(Canchor,CC): 
    if len(CC) == 0:
        return Canchor
    CC = eliminate_dup2(CC) 
    winner = CC[0]
    buf = Canchor
    for ele in CC:
        buf = buf + ele[0]
    return (Canchor+winner[0])

def find_sum(C):
    winner = (None,-1)
    runner_up = (None,-1)
    for i in range(len(C)):
        if C[i][-1] in ['and', 'but', 'for', 'nor', 'or', 'so', 'yet']:
            C[i] = C[i][:-1]
    for ele in C:
        if ele[1] > winner[1]:
            if winner[1] > runner_up[1]:
                runner_up = winner
            winner = ele
        elif ele[1] > runner_up[1]:
            runner_up = ele
    if(not winner[0] or not runner_up[0]):  
        return ("FAIL","FAIL")
    ans1 = ""
    for words in winner[0]:
        ans1 = ans1 + words[0] + " "
    ans1 = ans1[0].upper() + ans1[1:(len(ans1)-1)]
    ans2 = ""
    for words in runner_up[0]:
        ans2 = ans2 + words[0] + " "
    ans2 = ans2[0].upper() + ans2[1:(len(ans2)-1)]
    return ans1,ans2

def Traverse(clist,vk,score,PRI_overlap,sent,length):
    global G1
    global PRI1
    global sigma_r        
    redundancy = len(PRI_overlap)
    if length>10:
        return
    if vk in sent[:-1] and vk[1] in ['ADJ','ADV']:
        return
    if (redundancy  >= sigma_r): 
        if (VEN(vk)):
            if (valid_sent(sent)):
                finalscore = score/length
                clist.append((sent,finalscore))
                return 
        for vn in G1[vk]:
            PRI_new = PRI_overlap+check_inter(PRI_overlap,PRI1[vn]) #we use the second generation, but the old PRI is kept
            redundancy = len(PRI_new)                               #different from the paper
            new_sent = sent+[vn]
            L = length + 1
            new_score = score + redundancy 
            if (collapsible(vn)):
                Canchor = new_sent
                tmp = []
                for vx in G1[vn]:
                    if vx[1] not in ['.']:
                        Traverse(tmp,vx,0,PRI_new,[vx],L)
                    CC = eliminate_dup2(tmp) 
                    avg_CC_score = find_avg_score(CC)
                    finalscore = new_score + avg_CC_score
                    stitch_sent = stitch(Canchor,CC) 
                    if(valid_sent(stitch_sent)):
                        clist.append((stitch_sent,finalscore))
            else:
                Traverse(clist,vn,new_score,PRI_new,new_sent,L)

def first_lower(s):
    if s[0] == "I":
        return s[0] + s[1:].lower()
    else:
        return s.lower()
        
def read_text(file_name):
    redundant = '<br />'
    review = []
    with open(file_name,'r') as file:
        for article in file:
            article = article.replace(redundant," ")
            sentences = nltk.sent_tokenize(article)    
            for sentence in sentences:
                sentence = first_lower(sentence)
                review.append(nltk.pos_tag(nltk.word_tokenize(sentence),tagset='universal'))
                
    file.close()
    return review



In [7]:
import nltk
import os
filenames = [f for f in os.listdir('.') if os.path.isfile(f)]
for name in filenames: 
    a,b = name.split('.')
    if b == 'txt':     
        print(name)    
        Z1 = read_text(name)
        sigma_r = 2 
        G1 = dict()
        PRI1 = dict()
        OpinosisGraph(Z1, G1, PRI1)
        candidates = []
        for keys in G1:
            if(VSN(keys)):
                clist = []
                sum_sent = [keys]
                score = 0
                Traverse(clist,keys,score,PRI1[keys],sum_sent,1)
                candidates = candidates + clist
        C = eliminate_dup2(candidates)
        if len(C) < 2:
            sigma_r = 1
            for keys in G1:
                if(VSN(keys)):
                    clist = []
                    sum_sent = [keys]
                    score = 0
                    Traverse(clist,keys,score,PRI1[keys],sum_sent,1)
                    candidates = candidates + clist
        if len(C) < 2:
            sigma_r = 0
            for keys in G1:
                if(VSN(keys)):
                    clist = []
                    sum_sent = [keys]
                    score = 0
                    Traverse(clist,keys,score,PRI1[keys],sum_sent,1)
                    candidates = candidates + clist
        summary = find_sum(C)
        
        print("summary1: ",summary[0])
        print("summary2: ",summary[1])



00_9.txt
summary1:  The pettiness of the whole situation , the school , i
summary2:  The whole situation , the pettiness of the school , i
01_7.txt
summary1:  The idiotic principal , mr. bip , the small stories going
summary2:  The idiotic principal , mr. bip , tracy-ann oberman , smack
02_9.txt
summary1:  A students and teachers at a south london public school leaves
summary2:  Of keisha , of a better term , the show does
03_10.txt
summary1:  At the orchestra pit -- or even at the theatre stopped
summary2:  Is you and it needs more than your attention , it needs
04_10.txt
summary1:  The target of evil guys and `` the goon '' challenge
summary2:  Of the target of the very best three stooges shorts
05_10.txt
summary1:  In in in the middle of this film so i had
summary2:  In in in in in in south carolina , and danny
06_7.txt
summary1:  Movie , judging by the unusually low score imdb members have
summary2:  The writing and the use of this movie is the spanish gives
07_7.txt
summary1:  A g

In [ ]:
'''
modification:
1.use the second generation, but the technical error in the first generation is kept
Don't delete old PRI to ensure that Clists are not empty
problem:
1.Summaries are composed of some fragments of sentences(00_9.txt,01_7.txt)
2.The repeatnedss within a sentence is still unsolved
'''